In [2]:
import numpy as np
import scipy.stats as st
from matplotlib import pyplot as plt
from InformativeRandomSample import informative_sample
from Estimators import ACLEstimate, PREstimate, RREstimate
import pandas as pd

In [3]:
censorship = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
kernels = ['cosine', 'epanechnikov', 'gaussian', 'laplace', 'logistic', 'quartic', 'triangular', 'tricube', 'triweight', 'uniform']
bandwidths = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.23, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

In [4]:
import sqlite3
conn = sqlite3.connect('example500.db')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS example_table (
    value REAL,
    kernels TEXT,
    censorship REAL,
    bandwidths REAL
)
''')

In [5]:
n = 500

In [6]:
for kl in range(1200):
    for i in censorship:
        T, E = informative_sample(st.expon, n, n*i)
        t = ACLEstimate(T, E)
        x = np.delete(T, np.argmax(T))
        for j in bandwidths:
            for k in kernels:
                S = np.mean((t.pdf(x, bandwidth=j, kernel=k) - st.expon.pdf(x))**2)
                cursor.execute("""INSERT INTO example_table(value, kernels, censorship, bandwidths) VALUES (?, ?, ?, ?)""", (S, k, i, j))

/Users/dilshodmansurov/DataspellProjects/Bozorov/Kernels.py:364: RuntimeWarning: overflow encountered in cosh
  pdf_values = (0.5 / scale) / (1 + np.cosh(z))


In [7]:
conn.commit()
conn.close()

In [8]:
def sqlite_to_excel(df, kernel, censorships, bandwidths):
    A = np.array([])
    for a in bandwidths:
        for b in censorships:
            bb = df[(df.kernels == kernel) & (df.censorship == b) & (df.bandwidths == a)]['value'].mean()
            A = np.append(A, bb)
    return A.reshape(len(bandwidths), len(censorships))

In [67]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('example500.db')
cursor = conn.cursor()
query = "SELECT * FROM example_table"
df = pd.read_sql(query, conn)
conn.close()
df1 = sqlite_to_excel(df, 'tricube', [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6], [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.23, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
df2 = pd.DataFrame(df1, columns=['0', '10', '20', '30', '40', '50', '60'], index=bandwidths)

In [68]:
df2.to_excel('tricube500.xlsx')

In [69]:
conn.close()